# Instructions
Clone the repo with webkinz utils.

Put the game window in top left corner of your screen. This was tested on a Mac Book Pro with 'More Space' scaling in Display settings (2048x1280). You may need to fiddle with the offsets and game dims in the following cell. You can also change the CONSTANTS in the third cell to control game difficulty, the target score, etc.

Open the Lunch Letters home screen on WK.

Run all cells (imports, function definition, main game loop). Press '1' to quit.

In [12]:
import sys
sys.path.append('../common')
import utils
import importlib
importlib.reload(utils)

import time
import cv2
import string
import random 
import datetime
import hashlib
import time
import math

import numpy as np
import pynput.keyboard as kb

from enum import Enum
from collections import defaultdict
from pynput.keyboard import Key

%matplotlib inline
from matplotlib import pyplot as plt

io = utils.IOHandler(offset=(1081, 675), game_dims=(1200,900), verbose=True)

In [13]:
class StopMode(Enum):
  STOP_BY = 'Stop at or before' # Stop at or before the target score (useful for raking it in, without hitting the score cap)
  STOP_AT = 'Stop at exactly' # Stop exactly at the target score (useful for getting an unbeatable high score, equal to the cap)
  DONT_STOP = "Don't stop" # Don't stop the game, play until we lose (useful for bragging rights & seeing how high it can go)

class Screen(Enum):
    GAME_OVER = "game_over"
    HOME = "home"
    DIFFICULTY_VERY_HARD = "difficulty_very_hard"
    DIFFICULTY_MEDIUM = "difficulty_medium"
    DIFFICULTY_EASY = "difficulty_easy"
    DIFFICULTY_HARD = "difficulty_hard"
    LEVEL_COMPLETE = "level_complete"
    BE_CAREFUL = "be_careful"
    LEVEL_GO = "level_go"
    LEVEL_READY = "level_ready"
    OH_NO = "oh_no"
    GREAT_WORK = "great_work"
    CERTIFICATE = "certificate"
    ERROR_OCCURRED = "error_occurred"
    SCORE_EXCEEDS = "score_exceeds"
    GREAT_PLAY = "great_play"

SCREEN_TYPES = {
    Screen.GAME_OVER: [300, 90, [251, 235, 79]], # yellow "G"
    Screen.HOME: [600, 20, [128, 201, 250]], # light blue logo ring
    Screen.DIFFICULTY_VERY_HARD: [30, 108, [254, 254, 84]],
    Screen.DIFFICULTY_MEDIUM: [18, 124, [254, 254, 84]],
    Screen.DIFFICULTY_EASY: [34, 135, [254, 254, 84]],
    Screen.DIFFICULTY_HARD: [41, 125, [254, 254, 84]],
    Screen.LEVEL_COMPLETE: [600, 339, [188, 65, 30]], # top of red border
    Screen.BE_CAREFUL: [600, 280, [76, 160, 248]], 
    Screen.LEVEL_GO: [609, 355, [5, 13, 141]], # "o" in "Go!"
    Screen.LEVEL_READY: [577, 300, [5, 13, 141]], # "e" in "Ready"
    Screen.OH_NO: [608, 373, [2, 6, 93]], # "o" in "floor"
    Screen.GREAT_WORK: [260, 393, [2, 6, 93]], # "G" in "Great"
    Screen.CERTIFICATE: [190, 670, [219, 68, 51]], # Red "W" seal
    Screen.ERROR_OCCURRED: [210, 500, [33, 3, 137]], # Purple "W" outline
    Screen.SCORE_EXCEEDS: [390, 540, [33, 3, 137]], # Purple "W" outline
    Screen.GREAT_PLAY: [260, 690, [172, 68, 59]], # Red gift box lid
}

LETTER_MATCH_THRESHOLDS = {
  'little': {'A':0.85,'B':0.85,'C':0.9,'D':0.87,'E':0.9,'F':0.95,'G':0.9,'H':0.87,'I':0.9,'J':0.90,'K':0.87,'L':0.9,'M':0.85,'N':0.85,'O':0.94,'P':0.85,'Q':0.95,'R':0.87,'S':0.9,'T':0.9,'U':0.85,'V':0.92,'W':0.85,'X':0.92,'Y':0.85,'Z':0.90},
  'big': {'A':0.9,'B':0.9,'C':0.9,'D':0.9,'E':0.9,'F':0.9,'G':0.9,'H':0.9,'I':0.9,'J':0.9,'K':0.9,'L':0.9,'M':0.9,'N':0.9,'O':0.95,'P':0.9,'Q':0.94,'R':0.9,'S':0.9,'T':0.9,'U':0.9,'V':0.9,'W':0.9,'X':0.9,'Y':0.9,'Z':0.9}
}

def pad_image(image, W, H, value=255, middle_align=False, center_align=False):
  w, h, _ = image.shape
  if middle_align:
    pad_top = max(0, (H - h) // 2)
    pad_bot = max(0, H - h - pad_top)
  else:
    pad_top = 0
    pad_bot = max(0, H - h)

  if center_align:
    pad_lft = max(0, (W - w) // 2)
    pad_rt = max(0, W - w - pad_lft)
  else:
    pad_lft = 0
    pad_rt = max(0, W - w)

  return np.pad(image, ((pad_lft, pad_rt), (pad_top, pad_bot), (0, 0)), mode='constant', constant_values=value)

def save_letter_masks(in_dir, out_dir, w, h, color, center=False):
  letter_imgs = io.load_images_from_directory(in_dir)
  for letter_name, letter_img in letter_imgs.items():
    letter_pixels = (letter_img == color).all(axis=2)
    black_pixels = np.zeros_like(letter_img)
    white_pixels = np.ones_like(letter_img) * 255
    letter_mask = np.where(letter_pixels[:,:,None], black_pixels, white_pixels).astype(np.uint8)
    black_indices = np.where(letter_mask == 0)
    min_x, max_x = np.min(black_indices[0]), np.max(black_indices[0])
    min_y, max_y = np.min(black_indices[1]), np.max(black_indices[1])
    cropped_img = letter_mask[min_x:max_x+1, min_y:max_y+1, :]
    padded_img = pad_image(cropped_img, w, h, 255, center, center)
    io.save_image(padded_img, f"{out_dir}/{letter_name}")

def save_avg_letter_masks(in_dir, out_dir, crop=False):
  letter_masks = io.load_images_from_directory(in_dir)
  letter_imgs = defaultdict(list)
  for letter_name, letter_img in letter_masks.items():
    letter = letter_name[0]
    letter_imgs[letter].append(letter_img)
  for letter, letter_img_list in letter_imgs.items():
    avg_letter_img = utils.IOHandler.get_average_image(letter_img_list)
    if crop:
      color_indices = np.where(avg_letter_img != 255)
      min_x, max_x, min_y, max_y = np.min(color_indices[0]), np.max(color_indices[0]), np.min(color_indices[1]), np.max(color_indices[1])
      cropped_img = avg_letter_img[min_x:max_x+1, min_y:max_y+1, :]
      w, h, _ = cropped_img.shape
      padded_img = pad_image(cropped_img, w+2, h+2, 255, True, True)
      io.save_image(padded_img, f'{out_dir}/cropped/{letter}.png')
    io.save_image(avg_letter_img, f'{out_dir}/{letter}.png')

def get_and_image(letter_img_list):
  stacked_masks = np.stack(letter_img_list, axis=-1)
  output_mask = np.all(stacked_masks, axis=-1)
  return np.uint8(output_mask) * 255

def get_or_image(letter_img_list):
  stacked_masks = np.stack(letter_img_list, axis=-1)
  output_mask = np.any(stacked_masks, axis=-1)
  return np.uint8(output_mask) * 255

def save_reduced_letter_masks(in_dir, out_dir, method, crop=False):
  img_reducer = utils.IOHandler.get_average_image
  if method == 'and':
     img_reducer = get_and_image
  elif method == 'or':
     img_reducer = get_or_image

  letter_masks = io.load_images_from_directory(in_dir)
  letter_imgs = defaultdict(list)
  for letter_name, letter_img in letter_masks.items():
    letter = letter_name[0]
    letter_imgs[letter].append(letter_img)
  for letter, letter_img_list in letter_imgs.items():
    reduced_img = img_reducer(letter_img_list)
    if crop:
      color_indices = np.where(reduced_img != 255)
      min_x, max_x, min_y, max_y = np.min(color_indices[0]), np.max(color_indices[0]), np.min(color_indices[1]), np.max(color_indices[1])
      cropped_img = reduced_img[min_x:max_x+1, min_y:max_y+1, :]
      w, h, _ = cropped_img.shape
      padded_img = pad_image(cropped_img, w+2, h+2, 255, True, True)
      io.save_image(padded_img, f'{out_dir}/cropped/{letter}.png')
    io.save_image(reduced_img, f'{out_dir}/{letter}.png')

def save_avg_letter_masks(in_dir, out_dir, crop=False):
  return save_reduced_letter_masks(in_dir, out_dir, method='avg')

def save_or_letter_masks(in_dir, out_dir, crop=False):
  return save_reduced_letter_masks(in_dir, out_dir, method='or')

def save_and_letter_masks(in_dir, out_dir, crop=False):
  return save_reduced_letter_masks(in_dir, out_dir, method='and')

def save_deduped_letter_masks(in_dir, out_dir):
  letter_masks = io.load_images_from_directory(in_dir)
  letter_imgs = defaultdict(list)
  for letter_name, letter_img in letter_masks.items():
    letter = letter_name[0]
    hsh = hashlib.sha1(letter_img.tobytes()).hexdigest()
    letter_imgs[f'{letter}-{hsh}'].append(letter_img)
  for letter, letter_img_list in letter_imgs.items():
    avg_letter_img = utils.IOHandler.get_average_image(letter_img_list)
    io.save_image(avg_letter_img, f'{out_dir}/{letter}.png')

# Takes in an image (e.g. level_img) and creates a black/white mask of locations of the given color
def calc_color_mask(img, color):
  io_image = np.repeat(np.where((img == color).all(axis=2)[:, :, None], 0, 255), 3, axis=2)
  cv2_image = cv2.cvtColor(io_image.swapaxes(0, 1).astype(np.uint8), cv2.COLOR_RGB2BGR)
  return cv2_image, io_image

def get_match_template(level_mask, letter_mask):
  result = cv2.matchTemplate(level_mask, letter_mask, cv2.TM_CCOEFF_NORMED)
  return result

def get_all_rects(result, letter_mask, threshold=0.6):
  w, h, _ = letter_mask.shape
  yloc, xloc = np.where(result >= threshold)
  all_rects = []
  for (x, y) in zip(xloc, yloc):
      all_rects.append([int(x), int(y), int(w), int(h)])
  return all_rects

def plot_rects(img, rects, color=[255, 0, 0]):
  img = np.copy(img)
  for (x, y, w, h) in rects:
    cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
  return img

def plt_show_image(img):
  plt.imshow(img)
  plt.show()

def dedupe_rects(all_rects, threshold=0.2):
  rects = all_rects * 2
  rects, weights = cv2.groupRectangles(rects, 1, threshold)
  return rects, weights

def rand_color():
  return [random.randint(0, 255) for i in range(3)]

def load_letter_masks_cv2(crop=True):
  sizes = ['big', 'little']
  letter_masks = {}
  for size in sizes:
    if size not in letter_masks:
      letter_masks[size] = {}
      for letter in string.ascii_uppercase:
        crop_dir = ''
        if crop:
           crop_dir = 'cropped/'
        letter_mask = cv2.imread(f'letters/masks/{size}/centered/avg/{crop_dir}{letter}.png', cv2.IMREAD_UNCHANGED)
        letter_masks[size][letter] = letter_mask
  return letter_masks

def load_letter_masks():
  sizes = ['big', 'little']
  letter_masks = {}
  for size in sizes:
    if size not in letter_masks:
      letter_masks[size] = {}
      for letter in string.ascii_uppercase:
        letter_mask = io.load_image(f'letters/masks/{size}/centered/avg/{letter}.png')
        letter_masks[size][letter] = letter_mask
  return letter_masks

def get_letter_rects_from_image_old(letter_masks, level_mask, size="little", show_imgs=False, save_imgs=False):
  cummulative_img = np.copy(level_mask)
  letter_rects = defaultdict(list)
  for letter in string.ascii_uppercase:
    match_thresh = LETTER_MATCH_THRESHOLDS[size][letter]
    letter_mask = letter_masks[size][letter]
    # Calculate the match heatmap
    match_template = get_match_template(level_mask, letter_mask)
    # Find all locations of this letter
    all_rects = get_all_rects(match_template, letter_mask, match_thresh)
    # Dedupe similar rectangles
    deduped_rects, weights = dedupe_rects(all_rects)
    letter_rects[letter] += list(deduped_rects)

    if show_imgs or save_imgs:
      color = rand_color()
      all_rects_img = plot_rects(level_mask, deduped_rects, color)
      deduped_rects_img = plot_rects(level_mask, deduped_rects, color)
      cummulative_img = plot_rects(cummulative_img, deduped_rects, color)
      
      if show_imgs:
        plt_show_image(np.repeat(match_template[..., np.newaxis], 3, axis=-1))
        plt_show_image(all_rects_img)
        plt_show_image(deduped_rects_img)

      # if save_imgs:
      #   cv2.imwrite(f'saved/temp/{size}/{letter}.png', deduped_rects_img)
  
  # if save_imgs:
  #   cv2.imwrite(f'saved/temp/{size}/_all.png', cummulative_img)

  return letter_rects

# Given the bounding boxes of letters, group them into words (by looking at horizonal and vertical gaps)
# E.g. [{'letters': 'I', 'x': 45, 'y': 1, 'w': 30, 'h': 40}, {'letters': 'H', 'x': 0, 'y': 0, 'w': 30, 'h': 40}] --> [[{'letters': 'H', 'x': 0, 'y': 0, 'w': 30, 'h': 40}, {'letters': 'I', 'x': 45, 'y': 1, 'w': 30, 'h': 40}]]
def group_letters_into_words(letters):
    y_threshold = 3
    x_min, x_max = 11, 16
    # Sort the letters by y & x coordinates
    letters = sorted(letters, key=lambda l: (l['y'], l['x']))
    # Group the letters into lines
    lines = []
    current_line = []
    for i, letter in enumerate(letters):
        y_dist = abs(letter['y'] - letters[i-1]['y'])
        if i == 0 or y_dist <= y_threshold:
            current_line.append(letter)
        else:
            lines.append(current_line)
            current_line = [letter]
    lines.append(current_line)
    # Sort the letters in each line by x coordinate
    for line in lines:
        line.sort(key=lambda l: l['x'])
    # Group the letters in each line into words
    words = []
    for line in lines:
        if len(line) == 0:
           continue
        current_word = [line[0]]
        for i in range(1, len(line)):
            x_gap = abs(line[i]['x'] - current_word[-1]['x']) - current_word[-1]['w']
            if x_min <= x_gap <= x_max:
                current_word.append(line[i])
            else:
                words.append(current_word)
                current_word = [line[i]]
        words.append(current_word)
    return words

def get_words_from_capture_old(level_img, letter_masks, save_path=None):
  level_mask_big, _ = calc_color_mask(level_img, [0,0,0])
  level_mask_little, _ = calc_color_mask(level_img, [51,51,51])

  rects_little = get_letter_rects_from_image_old(letter_masks, level_mask_little, size="little", show_imgs=False, save_imgs=True)
  rects_big = get_letter_rects_from_image_old(letter_masks, level_mask_big, size="big", show_imgs=False, save_imgs=True)

  letters_little = list()
  for letter, rects in rects_little.items():
    letters_little += [{'letters': letter, 'x': rect[0], 'y': rect[1], 'w': rect[2], 'h': rect[3]} for rect in rects]
  letters_little = sorted(letters_little, key=lambda r: (r['y'], r['x']))

  single_letters = list()
  for letter, rects in rects_big.items():
    single_letters += [{'letters': letter, 'x': rect[0], 'y': rect[1], 'w': rect[2], 'h': rect[3]} for rect in rects]
  single_letters = sorted(single_letters, key=lambda r: (r['y'], r['x']))

  if save_path is not None:
     for l in letters_little:
        letter_img = io.crop_portion(l['x'], l['y'], l['w'], l['h'], level_img)
        # io.save_image(letter_img, f"{save_path}/little/{l['letters']}-{l['x']}-{l['y']}.png")
     for l in single_letters:
        letter_img = io.crop_portion(l['x'], l['y'], l['w'], l['h'], level_img)
        # io.save_image(letter_img, f"{save_path}/big/{l['letters']}-{l['x']}-{l['y']}.png")

  letters = letters_little.copy()
  words = group_letters_into_words(letters)
  words_rect_img = np.copy(level_mask_little)
  sandwich_words = []
  for word in words:
      minx, maxx, miny, maxy = min([x['x'] for x in word]), max([x['x'] for x in word]), min([x['y'] for x in word]), max([x['y'] for x in word])
      dx, dy = maxx - minx, maxy - miny
      wordw, wordh = dx + word[0]['w'], dy + word[0]['h']
      color = rand_color()
      words_rect_img = plot_rects(words_rect_img, [(x['x'], x['y'], x['w'], x['h']) for x in word], color)
      words_rect_img = plot_rects(words_rect_img, [(minx, miny, wordw, wordh)], color)
      word_str = ''.join([x['letters'] for x in word])
      sandwich_words.append({'letters': word_str, 'x': minx, 'y': miny, 'w': wordw, 'h': wordh})
      if save_path is not None:
        word_img = io.crop_portion(minx, miny, wordw, wordh, level_img)
        # io.save_image(word_img, f"{save_path}/words/{word_str}-{minx}-{miny}.png")
  
  # _ = cv2.imwrite(f'saved/temp/_words.png', words_rect_img)

  all_words = sandwich_words + single_letters
  all_words = sorted(all_words, key=lambda x: (-x['y'], -len(x['letters'])))

  return all_words

def click_level_button(difficulty):
  DIFFICULTY_BUTTON = {
     'easy': (200, 800),
     'medium': (450, 800),
     'hard': (750, 800),
     'very_hard': (1000, 800)
  }
  x, y = DIFFICULTY_BUTTON[difficulty]
  io.click_mouse(x, y)

def is_screen_type(screen_type, image=None):
    x, y, color = SCREEN_TYPES[screen_type]
    return io.is_given_screen(x, y, color, delta=10, image=image)
    
# TODO: could move to utils?
def calc_screen_types(image=None):
    return {screen_type for screen_type in SCREEN_TYPES.keys() if is_screen_type(screen_type, image)}

def save_game_cap(difficulty, game_img=None, filepath=None):
    if game_img is None:
        game_img = io.capture_screen()
    now = datetime.datetime.now()
    date_str = now.strftime("%Y-%m-%d %H-%M-%S")
    if filepath is None:
       filepath = f"saved/screenshots/level-X-{difficulty} {date_str}.png"
    io.save_image(game_img, filepath)

def parse_difficulty(level_img):
    screen_types = calc_screen_types(level_img)
    difficulty_map = {
        Screen.DIFFICULTY_VERY_HARD: 'very_hard',
        Screen.DIFFICULTY_MEDIUM: 'medium',
        Screen.DIFFICULTY_EASY: 'easy',
        Screen.DIFFICULTY_HARD: 'hard'
    }
    for screen_type, difficulty in difficulty_map.items():
        if screen_type in screen_types:
          return difficulty
    return None
      
def get_into_game(difficulty='very_hard'):
  global kill_program
  # Get to the main play screen
  while True:
    if kill_program:
      break
    game_img = io.capture_screen()
    screen_types = calc_screen_types(game_img)
    # print(screen_types)

    if Screen.GAME_OVER in screen_types:
      io.click_mouse(400, 700, delays=[0.3,0,0.3]) # "Replay" button
    elif Screen.HOME in screen_types:
      click_level_button(difficulty)
    elif screen_types.intersection(set([Screen.BE_CAREFUL, Screen.LEVEL_READY, Screen.LEVEL_GO])):
      pass
    elif Screen.OH_NO in screen_types:
      io.click_mouse(950, 770, delays=[0.3,0,0]) # "Close" button
    elif Screen.LEVEL_COMPLETE in screen_types:
      io.press_key(Key.enter)
    elif Screen.GREAT_WORK in screen_types:
      io.click_mouse(610, 770, delays=[0.3,0,0]) # "View Certificate" button
      io.click_mouse(700, 140, delays=[0.3,0,0.1]) # "Back" button
      io.click_mouse(950, 770, delays=[0.3,0,0.1]) # "Close" button
    elif Screen.CERTIFICATE in screen_types:
      io.click_mouse(700, 140, delays=[0.3,0,0]) # "Back" button
    elif Screen.ERROR_OCCURRED in screen_types:
      io.click_mouse(700, 580, delays=[0.3,0,0]) # "Ok" button
    elif Screen.SCORE_EXCEEDS in screen_types:
      io.click_mouse(700, 500, delays=[0.3,0,0]) # "Ok" button
    elif Screen.GREAT_PLAY in screen_types:
      io.click_mouse(600, 1000, delays=[0.3,0,0]) # "Collect" button
    elif screen_types.intersection(set([Screen.DIFFICULTY_VERY_HARD, Screen.DIFFICULTY_MEDIUM, Screen.DIFFICULTY_EASY, Screen.DIFFICULTY_HARD])):
      return

    time.sleep(0.5)

def on_press(key):
  global kill_program
  global listener
  if key == kb.KeyCode.from_char('1'):
    print("'1' pressed to quit")
    listener.stop()
    kill_program = True

def get_words_from_capture_new(level_img, letter_masks, save_path="saved/screenshots/_unorg"):
  level_mask_big_cv2, level_mask_big_io = calc_color_mask(level_img, [0,0,0])
  level_mask_little_cv2, level_mask_little_io = calc_color_mask(level_img, [51,51,51])
  
  rects_big = get_letter_rects_from_image_new(level_mask_big_io, level_mask_big_cv2, letter_masks, size="big")
  rects_little = get_letter_rects_from_image_new(level_mask_little_io, level_mask_little_cv2, letter_masks, size="little")

  letters_little = sorted(rects_little, key=lambda r: (r['y'], r['x']))
  single_letters = sorted(rects_big, key=lambda r: (r['y'], r['x']))

  if save_path is not None:
     for l in letters_little:
        letter_img = io.crop_portion(l['x'], l['y'], l['w'], l['h'], level_img)
        # io.save_image(letter_img, f"{save_path}/little/{l['letters']}-{l['x']}-{l['y']}.png")
     for l in single_letters:
        letter_img = io.crop_portion(l['x'], l['y'], l['w'], l['h'], level_img)
        # io.save_image(letter_img, f"{save_path}/big/{l['letters']}-{l['x']}-{l['y']}.png")

  letters = letters_little.copy()
  words = group_letters_into_words(letters)
  words_rect_img = np.copy(level_mask_little_cv2)
  sandwich_words = []
  for word in words:
      minx, maxx, miny, maxy = min([x['x'] for x in word]), max([x['x'] for x in word]), min([x['y'] for x in word]), max([x['y'] for x in word])
      dx, dy = maxx - minx, maxy - miny
      wordw, wordh = dx + word[0]['w'], dy + word[0]['h']
      color = rand_color()
      words_rect_img = plot_rects(words_rect_img, [(x['x'], x['y'], x['w'], x['h']) for x in word], color)
      words_rect_img = plot_rects(words_rect_img, [(minx, miny, wordw, wordh)], color)
      word_str = ''.join([x['letters'] for x in word])
      sandwich_words.append({'letters': word_str, 'x': minx, 'y': miny, 'w': wordw, 'h': wordh})
      if save_path is not None:
        word_img = io.crop_portion(minx, miny, wordw, wordh, level_img)
        # io.save_image(word_img, f"{save_path}/words/{word_str}-{minx}-{miny}.png")
  
  # _ = cv2.imwrite(f'saved/temp/_words.png', words_rect_img)

  all_words = sandwich_words + single_letters
  all_words = sorted(all_words, key=lambda x: (-x['y'], -len(x['letters'])))

  return all_words

def get_letter_rects_from_image_new(level_mask, level_mask_cv2, all_letter_masks, size='little'):
  letter_masks = all_letter_masks[size]
  mask = cv2.cvtColor(level_mask_cv2, cv2.COLOR_BGR2GRAY) 

  contours, _ = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
  img_contours = mask.copy()
  img_contours = cv2.cvtColor(img_contours, cv2.COLOR_GRAY2BGR)
  cv2.drawContours(img_contours, contours, -1, (0, 0, 255), 1)

  max_dist = 170

  mask_w, mask_h = 30, 30
  min_a, max_a = 170, 790
  min_w, max_w = 6, 31
  min_h, max_h = 25, 31
  if size == 'big':
    mask_w, mask_h = 38, 38
    min_a, max_a = 170, 1444
    min_w, max_w = 6, 38
    min_h, max_h = 29, 38

    
  letter_squares = []
  # Draw bounding rectangles around contours
  for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    pad_top = max(0, (mask_h - h) // 2)
    pad_left = max(0, (mask_w - w) // 2)
    x2, y2 = x - pad_left, y - pad_top
    area = w * h
    if min_a <= area <= max_a and min_w <= w <= max_w and min_h <= h <= max_h:
      cv2.rectangle(img_contours, (x, y), (x+w, y+h), (0, 255, 0), 1)
      cv2.rectangle(img_contours, (x2, y2), (x2+mask_w, y2+mask_h), (255, 0, 0), 2)
      letter_mask = io.crop_portion(x2, y2, mask_w, mask_h, level_mask)
      best_mask_idx, dist = utils.IOHandler.get_best_mask(letter_mask, letter_masks.values())
      best_letter = list(letter_masks.keys())[best_mask_idx]
      letter_squares.append({'letters': best_letter, 'x': x2, 'y': y2, 'w': mask_w, 'h': mask_h})
      # best_letter_img = list(letter_masks.values())[best_mask_idx]
      # print(size, best_letter, dist, w, h, int(area))
      if dist > max_dist:
        print("WARNING no match found")
      # io.show_image(best_letter_img)
      # io.show_image(letter_mask)
  # cv2.imwrite(f'saved/temp/_bounding_boxes.png', img_contours)
  # plt_show_image(img_contours)
  return letter_squares

def parse_score_img(score_img, digit_masks):
  digit_str = ''
  tot_dist = 0
  for i in range(6):
    x = round(i*(15+1/3))
    w, h = 16, 23
    digit_img = io.crop_portion(x, 0, w, h, score_img)
    best_digit_idx, dist = utils.IOHandler.get_best_mask(digit_img, digit_masks.values())
    best_digit_name = list(digit_masks.keys())[best_digit_idx]
    if dist > 35:
      print(f"WARNING: digit dist is high: '{best_digit_name[:-4]}' ({dist})")
    tot_dist += dist
    if best_digit_name.startswith('blank'):
      digit_str += ' '
    else:
      digit_str += best_digit_name[0]
  return int(digit_str), tot_dist / 6

def parse_score_from_level_img(level_img, digit_masks):
  score_img = io.crop_portion(150, 33, 93, 23, level_img)
  # plt_show_image(score_img.swapaxes(0,1))
  return parse_score_img(score_img, digit_masks)

def parse_level_from_level_img(level_img, digit_masks):
  lvl_img = io.crop_portion(867, 33, 32, 23, level_img)
  # plt_show_image(lvl_img.swapaxes(0,1))
  return parse_level_from_lvl_img(lvl_img, digit_masks)

def parse_level_from_lvl_img(lvl_img, digit_masks):
  centered_str = ''
  centered_dist = 0
  rt_align_str = ''
  rt_align_dist = 0
  for i in range(3):
    x = 8*i
    w, h = 16, 23
    digit_img = io.crop_portion(x, 0, w, h, lvl_img)
    best_digit_idx, dist = utils.IOHandler.get_best_mask(digit_img, digit_masks.values())
    best_digit_name = list(digit_masks.keys())[best_digit_idx]
    if best_digit_name.startswith('blank'):
      if i == 1:
        centered_str += ' '
        centered_dist += dist
      else:
        rt_align_str += ' '
        rt_align_dist += dist
    else:
      if i == 1:
        centered_str += best_digit_name[0]
        centered_dist += dist
      else:
        rt_align_str += best_digit_name[0]
        rt_align_dist += dist
  rt_align_dist = rt_align_dist / 2
  if centered_dist * 2 < rt_align_dist:
    if centered_dist > 35:
      print(f"WARNING: centered digit dist is high: '{centered_str}' ({centered_dist})")
    return int(centered_str), centered_dist
  else:
    if rt_align_dist > 35:
      print(f"WARNING: rt. aligned digit dist is high: '{rt_align_str}' ({rt_align_dist})")
    return int(rt_align_str), rt_align_dist
  
def calc_word_score(word, level):
  word_len = len(word)
  if word_len == 1:
    est_points = 10 + level // 5
  else:
    pts_per_let = 7.55 + 0.15*level
    est_points = int(pts_per_let*word_len)
  return est_points

def calc_penalty(level):
  return int(29.4 + level*0.6)

def calc_desired_penalties(curr_score, target_score, level):
  single_pts = calc_word_score('A', level)
  penalty_pts = calc_penalty(level)
  for num_singles in range(10):
    num_penalties = (single_pts*num_singles - (curr_score - target_score)) / penalty_pts
    if num_penalties.is_integer():
      return num_penalties
  num_penalties = (curr_score - target_score) / penalty_pts
  if num_penalties.is_integer():
    return num_penalties
  return math.ceil(num_penalties)

# Takes little screenshots we have of the big & little letters & creates centered masks for each, which we then deduplicate and average
def regenerate_letter_masks():
  save_letter_masks('letters/raw/big', 'letters/masks/big/centered/all', 38, 38, [0, 0, 0], center=True)
  save_letter_masks('letters/raw/little', 'letters/masks/little/centered/all', 30, 30, [51, 51, 51], center=True)

  save_deduped_letter_masks('letters/masks/big/centered/all', 'letters/masks/big/centered/deduped')
  save_deduped_letter_masks('letters/masks/little/centered/all', 'letters/masks/little/centered/deduped')

  save_avg_letter_masks('letters/masks/big/centered/deduped', 'letters/masks/big/centered/avg')
  save_avg_letter_masks('letters/masks/little/centered/deduped', 'letters/masks/little/centered/avg')

# This tests how well our letter parsing is working, using a screenshot that has a bunch of words/letters pasted onto it
def test_abom_image():
  letter_masks = load_letter_masks()
  level_img = io.load_image('./saved/level-abomination.png')
  solution = get_words_from_capture_new(level_img, letter_masks, save_path=None)
  solution = sorted(solution, key=lambda x: (len(x['letters']) != 1, -x['y']))
  print(', '.join([word['letters'] for word in solution]))

# TODO: couldnt_be_broadcast_together.png has issue because apple very close to edge of screen
def test_couldnt_be_broadcast_together():
  level_img = io.load_image('./couldnt_be_broadcast_together.png')
  io.show_image(level_img)
  solution = get_words_from_capture_new(level_img, letter_masks, save_path="saved/screenshots/_unorg")

In [14]:
TARGET_SCORE = 60_000
STOP_MODE = StopMode.STOP_BY
DIFFICULTY = 'very_hard'
NUM_GAMES = -1 # TODO: add num_games parameter

kill_program = False
listener = kb.Listener(on_press=on_press)
listener.start()

# Can simplify by just using one of the level masks
# TODO: Make a common util for parsing numbers
digit_masks = io.load_images_from_directory('./digits')
digit_masks = {name: mask for name, mask in digit_masks.items() if name.endswith('_h.png') or name == 'blank.png'}
letter_masks = load_letter_masks()

running_score = 0
in_point_losing_mode = False
# Run game loop
while True:
  if kill_program:
      break
  io.click_mouse(0, 0) # Focus Webkinz
  get_into_game(DIFFICULTY)

  level_img = io.capture_screen()
  curr_score, _ = parse_score_from_level_img(level_img, digit_masks)
  level, _ = parse_level_from_level_img(level_img, digit_masks)
  running_score = curr_score

  # If we want an exact score, use penalties to fall at/below target
  if STOP_MODE == StopMode.STOP_AT and curr_score >= TARGET_SCORE:
    if curr_score == TARGET_SCORE:
      print(f"Stop current game - reached the target score ({TARGET_SCORE})")
      io.click_mouse(1160, 50, delays=[0.3,0,0]) # "x" button
      continue
    penalty = calc_penalty(level)
    desired_penalties = calc_desired_penalties(curr_score, TARGET_SCORE, level)
    while running_score > TARGET_SCORE:
      if not in_point_losing_mode:
        for i in range(5):
          io.press_key('0', delays=[0,0])
        in_point_losing_mode = True
      io.press_key('0', delays=[0,0])
      running_score -= penalty
  
  # If we have a point cap, stop before we exceed it
  if STOP_MODE == StopMode.STOP_BY:
    min_word_pts = calc_word_score('A', level)
    if curr_score + min_word_pts > TARGET_SCORE:
      # print(f"Stop current game - {curr_score} is {TARGET_SCORE - curr_score} point(s) (<= {min_word_pts}) below target score ({TARGET_SCORE})")
      io.click_mouse(1160, 50, delays=[0.3,0,0]) # "x" button
      continue

  solution = get_words_from_capture_new(level_img, letter_masks, save_path="saved/screenshots/_unorg")
  solution = sorted(solution, key=lambda x: (len(x['letters']) != 1, -x['y']))
  soln_str = ', '.join([word['letters'] for word in solution])

  # print(f"{level}, {curr_score}, '{soln_str}'")

  for word in solution:
    letters = word['letters']
    if STOP_MODE in [StopMode.STOP_AT, StopMode.STOP_BY]:
      word_pts = calc_word_score(letters, level)
      if running_score == TARGET_SCORE:
        break
      elif STOP_MODE == StopMode.STOP_BY and running_score + word_pts > TARGET_SCORE:
        # print(f"Skipping '{letters}' because {running_score} + {word_pts} > {TARGET_SCORE}")
        continue
      else:
        running_score += word_pts
        in_point_losing_mode = False
    for letter in letters:
      io.press_key(letter, delays=[0, 0])
    io.press_key(' ', delays=[0, 0])

listener.stop()

'1' pressed to quit


## TODO:
- Could make the parsing of the letters/words faster
- Clean up old screenshots to save some storage
- Add to github
    - Figure out requirements.txt
    - Add common/utils repo
- Add to website
- Record a long playthrough
- Fix couldn't be broadcast together error (see test_couldnt_be_broadcast_together())
- Make util for parsing numbers

## Notes
- ~$480 / game (each is ~10.83 minutes) -> ~$2653/hour